# Assignment: Analyzing Airline Flight Delays 
#### By Brett Hallum, Chris Ficklin, and Ryan Shuhart<br>April 2017

For a full treatment of the unit 14 case study, please review module 14.3. Some points from the video are given below.

Work with the airline data set (use R or Python to manage out-of-core).
Answer the following questions by using the split-apply-combine technique:
* Which airports are most likely to be delayed flying out of or into?
* Which flights with same origin and destination are most likely to be delayed?
* Can you regress how delayed a flight will be before it is delayed?
* What are the most important features for this regression?

Remember to properly cross-validate models.

Use meaningful evaluation criteria.

Create at least one new feature variable for the regression.

In [1]:
import dask.dataframe as dd #http://dask.pydata.org/en/latest/
import pandas as pd
import numpy as np
from datetime import datetime
from bokeh.io import output_notebook

# from dask.distributed import Client
# client = Client(set_as_default=True)
# print(client)

### Other Settings
# Show more rows
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

# Prevent scientific notation of decimals
pd.set_option('precision',3)
pd.options.display.float_format = '{:,.3f}'.format

In [2]:
# Allow inline display of bokeh graphics
output_notebook()

Loading BokehJS ...

## [Here is some info about Dask]...

...General facts about Dask... blah blah

## Data

In [3]:
# http://stat-computing.org/dataexpo/2009/the-data.html
var_desc = pd.read_csv("../ref/var_descriptions.csv", index_col='var_id')
var_desc

,Name,Data Type,Description
var_id,,,
1,Year,int64,1987-2008
2,Month,int64,1 - 12
3,DayofMonth,int64,1 - 31
4,DayOfWeek,int64,1 (Monday) - 7 (Sunday)
5,DepTime,float64,"actual departure time (local, hhmm)"
6,CRSDepTime,int64,"scheduled departure time (local, hhmm)"
7,ArrTime,float64,"actual arrival time (local, hhmm)"
8,CRSArrTime,int64,"scheduled arrival time (local, hhmm)"
9,UniqueCarrier,O,unique carrier code


In [4]:
# Data Location
#parq_folder = "../data/parquet-tiny/" # Testing
parq_folder = "../data/parquet_25/" # Higher Load Testing
#parq_folder = "../data/parquet/" # Full data

# Load compressed Parquet format of all years ~2 sec
start = datetime.now()
df = dd.read_parquet(parq_folder)
print("Load parquet time: ", datetime.now() - start)
print()

# Length of dask dataframe ~3 min
start = datetime.now()
print("There are {:,d} rows".format(len(df))) #123,534,969 Matches Eric Larson
print("Time to determine row count: ", datetime.now() - start)

Load parquet time:  0:00:00.680573

There are 30,883,750 rows
Time to determine row count:  0:00:29.938199


### Glance at Beginning and End

In [5]:
print("First 5 rows:")
df.head()

First 5 rows:


,Year,Month,DayOfWeek,DepTime,CRSDepTime,UniqueCarrier,TailNum,ArrDelay,DepDelay,Origin,Dest,Distance
0,1987,10,5,"1,814.000",1815,CO,None,2.000,-1.000,ICT,IAH,542.000
1,1987,11,2,"1,328.000",1318,PI,None,12.000,10.000,CLT,TYS,177.000
2,1987,10,5,645.000,645,CO,None,-10.000,0.000,ORD,DEN,888.000
3,1987,10,4,"1,255.000",1245,NW,None,14.000,10.000,SFO,MEM,"1,807.000"
4,1987,10,7,800.000,800,DL,None,24.000,0.000,LAX,HNL,"2,556.000"


In [6]:
print("Last 5 rows:")
df.tail()

Last 5 rows:


,Year,Month,DayOfWeek,DepTime,CRSDepTime,UniqueCarrier,TailNum,ArrDelay,DepDelay,Origin,Dest,Distance
124799,2008,12,5,"1,225.000",1105,9E,88919E,79.000,80.000,DSM,MEM,490.000
124800,2008,12,1,"1,216.000",1220,WN,N645SW,3.000,-4.000,MCO,BUF,"1,011.000"
124801,2008,12,3,933.000,935,DL,N908DA,-11.000,-2.000,SLC,SAN,626.000
124802,2008,12,7,nan,858,YV,N27185,nan,nan,ORD,GRR,137.000
124803,2008,12,2,726.000,730,AS,N581AS,-8.000,-4.000,DFW,SEA,"1,660.000"


## Feature Preparation and Creation

In [7]:
def scaler(df, column):
    return (df[column] - df[column].mean())/df[column].std()

# Create an hour field
# 2400 minutes from midnight reduced to 2399 then int division drops to 23
df = df.assign(Hour=df.CRSDepTime.clip(upper=2399)//100) 

# Make Categories as categorical
df = df.categorize(['DayOfWeek', 'UniqueCarrier', 'Dest', 'Origin'])

# Months from 0 AD
df['FlightAge'] = 12*df['Year']+df['Month']-1

# The months from the first recorded flight is consider the approx age of the plane. 
# Unfortunately, tail numbers not tracked until 1995. 

# Find the first year and month of a tail numbers flight history
tail_births = (df.groupby('TailNum')[['FlightAge']].min().reset_index()
                 .rename(columns={'FlightAge':'FirstFlight'}))

df_with_tails = dd.merge(df, tail_births, how='left', on='TailNum')
df_with_tails['Age'] = df_with_tails['FlightAge'] - df_with_tails['FirstFlight']

df_with_tails = df_with_tails.drop(['FlightAge','FirstFlight'], axis=1)


# Scale columns for regression of all data
df['Hour_scaled'] = scaler(df, 'Hour')
df['Distance_scaled'] = scaler(df, 'Distance')

# Scale columns for regression for after 1994
df_with_tails['Hour_scaled'] = scaler(df_with_tails, 'Hour')
df_with_tails['Distance_scaled'] = scaler(df_with_tails, 'Distance')
df_with_tails['Age_scaled'] = scaler(df_with_tails, 'Age')

## Flight Delays

When a schedule airflight is behind more than 15 minutes then it is officially delayed. Same logic will be followed for arrival times. Only arrivals 15 minutes past scheduled time will be considered late

http://aspmhelp.faa.gov/index.php/Types_of_Delay

### Aggregations

View visualization of dask distrubuted at work

http://127.0.0.1:8787/

In [8]:
import dask
start = datetime.now()
# Define some aggregations to plot
aggregations = (
    #1 Average departure delay by year
    df.groupby('Year').DepDelay.mean(),
    
    #2 Average departure delay by Month
    df.groupby('Month').DepDelay.mean(), 
    
    #3 Average departure delay by hour of day
    df.groupby('Hour').DepDelay.mean(), 
    
    #4 Average departure delay by Carrier, top 15
    df.groupby('UniqueCarrier').DepDelay.mean().nlargest(15), 
    
    #5 Average arrival delay by destination, top 15
    (df.groupby('Dest').ArrDelay.mean().nlargest(15) 
     .reset_index().rename(columns={'ArrDelay':'AvgArrDelay'})),
    
    #6 Count of arrivals to destinations, excludes missing
    (df.groupby('Dest').ArrDelay.count() 
     .reset_index().rename(columns={'ArrDelay':'ArrCount'})),
    
    #7 Average departure delay by origin, top 15
    (df.groupby('Origin').DepDelay.mean().nlargest(15).reset_index().rename(columns={'DepDelay':'AvgDepDelay'})),
    
    #8 Count of departures by origin, excludes missing
    (df.groupby('Origin').DepDelay.count().reset_index().rename(columns={'DepDelay':'DepCount'})), 
    
    #9 Average departure by origin and destination
    (df.groupby(['Origin','Dest']).DepDelay.mean().reset_index().rename(columns={'DepDelay':'AvgDepDelay'})),
    
    #10 Count of departures between origin and destination
    (df.groupby(['Origin','Dest']).DepDelay.count().reset_index().rename(columns={'DepDelay':'DepCount'})),
    
    #11 Percentage of officially delayed flights by origin
    ((df[df.DepDelay>15].groupby('Origin').DepDelay.count() / df.groupby('Origin').DepDelay.count())
     .reset_index().rename(columns={'DepDelay':'PercDepDelay'})),
    
    #12 Percentage of officially late flights by destination
    ((df[df.ArrDelay>15].groupby('Dest').ArrDelay.count() / df.groupby('Dest').ArrDelay.count())
     .reset_index().rename(columns={'ArrDelay':'PercArrDelay'})),
                
    #13 Percentage of officially delayed flights by origin and destination
    ((df[df.DepDelay>15].groupby(['Origin','Dest']).DepDelay.count() / df.groupby(['Origin','Dest']).DepDelay.count())
     .reset_index().rename(columns={'DepDelay':'PercDepDelay'})),
                
    #14 Percentage of officially late flights by origin and destination
    ((df[df.ArrDelay>15].groupby(['Origin','Dest']).ArrDelay.count() / df.groupby(['Origin','Dest']).ArrDelay.count())
     .reset_index().rename(columns={'ArrDelay':'PercArrDelay'})),
    
    #15 Average departure delay by hour of day
    df.groupby('DayOfWeek').DepDelay.mean()
)

# Compute them all in a single command
(
delayed_by_year, #1
delayed_by_month, #2
delayed_by_hour, #3
delayed_by_carrier, #4
delayed_by_dest, #5
delayed_by_dest_count, #6
delayed_by_origin, #7
delayed_by_origin_count, #8
delayed_by_origin_dest, #9
delayed_by_origin_dest_count, #10
pct_delayed_by_origin, #11
pct_late_by_dest, #12
pct_delayed_by_origin_dest, #13
pct_late_by_origin_dest, #14
delayed_by_day #15
) = dask.compute(*aggregations)
print(datetime.now() - start)

0:01:13.640850


### Visualization of Average Delay

In [9]:
from bokeh.plotting import figure, show
from bokeh.charts.attributes import cat
from bokeh.charts import Bar
from bokeh.layouts import gridplot

# Average Delay by Year
p1 = Bar(delayed_by_year.reset_index(), 'Year', values= 'DepDelay', 
         legend=False, ylabel="Average Delay in Minutes", 
         title="Average Delay by Year")

# Average Delay by Month
delayed_by_month = delayed_by_month.sort_index()
p2 = Bar(delayed_by_month.reset_index(), 'Month', values= 'DepDelay', 
         legend=False, ylabel="Average Delay in Minutes", 
         title="Average Delay by Month")

# Average Delay by Hour of Day
p3 = Bar(delayed_by_hour.reset_index(), 'Hour', values= 'DepDelay', 
         legend=False, ylabel="Average Delay in Minutes",
         title="Average Delay by Hour of Day")

# Average Delay by Hour of Day
p4 = Bar(delayed_by_day.reset_index(), 'DayOfWeek', values= 'DepDelay', 
         legend=False, ylabel="Average Delay in Minutes",
         title="Average Delay by Day of Week")

# Average Delay by Carrier
delayed_by_carrier = delayed_by_carrier.reset_index()
delayed_by_carrier['UniqueCarrier'] = delayed_by_carrier['UniqueCarrier'].astype('O')
p5 = Bar(delayed_by_carrier, label=cat('UniqueCarrier', sort=False), values= 'DepDelay', 
         legend=False, ylabel="Average Delay in Minutes", xlabel="Unique Carrier", title="Average Delay by Carrier")


show(gridplot([[p1,p2],[p3,p4], [p5,None]], plot_width=400, plot_height=300))

## Which airports are most likely to be delayed flying out of or into?

In [10]:
airport_delays_pcts = (pd.merge(pct_delayed_by_origin, pct_late_by_dest, left_on='Origin', right_on='Dest')
                 .assign(AvgDelay= lambda x: (x['PercDepDelay'] + x['PercArrDelay'])/2)
                 .sort_values(by='AvgDelay', ascending=False)
                 .drop('Dest', axis=1)
                )

airport_delays_pcts = pd.merge(airport_delays_pcts, delayed_by_origin_count, on='Origin')

airport_delays_pcts[airport_delays_pcts['DepCount'] > 50].nlargest(15, 'AvgDelay')

,Origin,PercDepDelay,PercArrDelay,AvgDelay,DepCount
0,SOP,0.441,0.407,0.424,68
1,ADK,0.496,0.291,0.394,125
2,OTH,0.320,0.406,0.363,122
3,ACK,0.297,0.390,0.343,472
5,HHH,0.218,0.365,0.292,435
6,DUT,0.296,0.273,0.285,1222
7,ILG,0.211,0.335,0.273,185
8,LMT,0.286,0.252,0.269,133
9,PSG,0.230,0.272,0.251,3595
10,MQT,0.153,0.348,0.251,1427


## Which flights with same origin and destination are most likely to be delayed?

In [11]:
org_dest_pcts = (pd.merge(pct_delayed_by_origin_dest, pct_late_by_origin_dest, on=['Origin','Dest'])
                 .assign(AvgDelay= lambda x: (x['PercDepDelay'] + x['PercArrDelay'])/2)
                 .sort_values(by='AvgDelay', ascending=False)
                )

org_dest_pcts = pd.merge(org_dest_pcts, delayed_by_origin_dest_count, on=['Origin','Dest'])

org_dest_pcts[org_dest_pcts['DepCount'] > 50].nlargest(15, 'AvgDelay')

,Origin,Dest,PercDepDelay,PercArrDelay,AvgDelay,DepCount
89,IDA,ORD,0.614,0.565,0.590,70
92,BTR,EWR,0.500,0.643,0.571,56
97,ASE,ORD,0.544,0.532,0.538,125
99,ADK,AKN,0.543,0.523,0.533,94
100,EWR,JAN,0.448,0.596,0.522,58
131,CMH,BDL,0.490,0.510,0.500,51
136,EWR,LIT,0.430,0.491,0.460,435
137,SOP,ATL,0.441,0.471,0.456,68
140,IAD,BHM,0.389,0.503,0.446,301
141,ATL,BUR,0.358,0.533,0.445,123


In [12]:
from bokeh.charts import Histogram

hist = Histogram(df[df['DepDelay']>15][['DepDelay']].sample(.25).compute().dropna(), 
                 values='DepDelay', bins=50)

show(hist)

## Can you regress how delayed a flight will be before it is delayed?

## What are the most important features for this regression?

# Regression of Delay

The Dask module is a solution for processing "big data," however, the it currently does not include built in methods for regression or classification, like other big data solutions. The following will use a series of simple random sampling to a size that fits into a pandas dataframe to find the coefficient estimates of a linear model. The coefficients will be averaged to make a final prediction. This process also assists in not over fitting the model.

#### The following features will be explore to predict if the flight will have departure delay

##### The predicted variable will be: 
* Departure Delay (DepDelay)

##### The explanatory variables:
* Scheduled departure hour (Hour)
* Flight distance (Distance)
* Age of plane (Age)

In [8]:
# Sample the entire data set as large as possible a few times. Each time has it's own cross validation sampling.
def sample_coef(Xcols, ycol, df, samp_size = .1, seeds = [123,456,789,101,112]):
    t = datetime.now()
    from sklearn import linear_model
    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import r2_score
    import dask
    reg = linear_model.LinearRegression(n_jobs=-1)
    coefs = []
    
    for i in range(len(seeds)):
        start = datetime.now()
        # Take a sample from all the data
        all_cols = [ycol] + Xcols
        Xy = df[all_cols].sample(samp_size, random_state=seeds[i]).compute().dropna(axis=0)
        X = Xy[Xcols]
        y = Xy[ycol].values

        reg.fit(X, y)
        #print('Coefficients: \n', reg.coef_)
        coefs.append(reg.coef_)
        print("Time for Sample {}: ".format((i+1)), datetime.now() - start)
        #print(datetime.now() - start)
    
    del Xy, X, y

    coef_df = pd.DataFrame.from_records(coefs, columns=Xcols)
    coef_avg = coef_df.mean()
    print("\nCoefficients:")
    print(coef_df)    
    print("\nAverage Coefficients:")
    print(coef_avg)
    
    beta_cols = []
    for m, c in zip(coef_avg.index, coef_avg.values):
        b_col = "Beta_"+m
        df["Beta_"+m] = df[m]*c
        beta_cols.append(b_col)

    df['Predicted'] = df[beta_cols].sum(axis=1)

    #df['SqError'] = (df['Predicted'] - df[ycol])**2
    #mse = df[['SqError']].mean().compute()
    
    df_tmp = df[['Predicted']+[ycol]].sample(.4).compute().dropna()
    y_true = df_tmp[ycol]
    y_pred = df_tmp['Predicted']
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print("\nEstimated Evaluation Scores on 40%:")
    print("Mean Squared Error: ", mse)
    print("R Squared: ", r2)
    print("\nTotal Time: ", datetime.now() - t)
    return coef_df, coef_avg

In [9]:
Xcols = ['Hour', 'Distance']
ycol =  'DepDelay'
coef_df, coef_avg = sample_coef(Xcols, ycol, df)

Time for Sample 1:  0:00:59.291417
Time for Sample 2:  0:00:59.258690
Time for Sample 3:  0:00:59.570244
Time for Sample 4:  0:00:59.393964
Time for Sample 5:  0:00:59.213273

Coefficients:
   Hour  Distance
0 0.723     0.001
1 0.726     0.001
2 0.731     0.001
3 0.730     0.001
4 0.726     0.001

Average Coefficients:
Hour       0.727
Distance   0.001
dtype: float64

Estimated Evaluation Scores on 40%:
Mean Squared Error:  814.49861276
R Squared:  0.00823510007062

Total Time:  0:06:02.475493


In [10]:
Xcols = ['Hour_scaled', 'Distance_scaled']
ycol =  'DepDelay'
coef_df, coef_avg = sample_coef(Xcols, ycol, df)

Time for Sample 1:  0:01:04.912052
Time for Sample 2:  0:01:04.738038
Time for Sample 3:  0:01:04.821900
Time for Sample 4:  0:01:04.799781
Time for Sample 5:  0:01:05.089071

Coefficients:
   Hour_scaled  Distance_scaled
0        3.437            0.821
1        3.452            0.810
2        3.474            0.797
3        3.469            0.805
4        3.452            0.811

Average Coefficients:
Hour_scaled       3.457
Distance_scaled   0.809
dtype: float64

Estimated Evaluation Scores on 40%:
Mean Squared Error:  869.851440569
R Squared:  -0.0666649236014

Total Time:  0:06:36.698518


In [ ]:
Xcols = ['Hour', 'Distance', 'Age']
ycol =  'DepDelay'
coef_df, coef_avg = sample_coef(Xcols, ycol, df_with_tails)

Time for Sample 1:  0:01:23.084160
Time for Sample 2:  0:01:22.715566
Time for Sample 3:  0:01:23.024373
Time for Sample 4:  0:01:22.117111
Time for Sample 5:  0:01:23.487107

Coefficients:
   Hour  Distance   Age
0 0.827     0.001 0.004
1 0.827     0.001 0.004
2 0.835     0.001 0.003
3 0.833     0.001 0.003
4 0.828     0.001 0.004

Average Coefficients:
Hour       0.830
Distance   0.001
Age        0.004
dtype: float64

Estimated Evaluation Scores on 40%:
Mean Squared Error:  815.97788893
R Squared:  -0.000646969409774

Total Time:  0:08:28.939244


In [ ]:
Xcols = ['Hour_scaled', 'Distance_scaled', 'Age_scaled']
ycol =  'DepDelay'
coef_df, coef_avg = sample_coef(Xcols, ycol, df_with_tails)

Time for Sample 1:  0:01:33.123734


### Conclusion

Dask is a new "big data" alternative for those preferring the Python language. Although it is in active development by Continuum.io it still lacks certain features, such as, a drop-in generalized linear model. The dask dataframe is easy to use as it follow the popular pandas convention, however, unfortunately it does not allow for row slicing. Row slicing is convient for mini-batch operations that could have been usefull in this case. It is possible to do row slicing with dask arrays, but the data must be on disk using a compatible storage. Parquet is not compatible. Therefore, at this stage of the dask project, dask dataframes work well with out-of-core classification machine learning, and dask arrays are suitable for out-of-core regression for techniques that can utilize mini-batch operations. The dask-glm project looks to tackle out-of-core and distributed regression problems.

### Future Work

* Optimize with index key base on Data, deptarture time, and TailNum
* Use of alternative compression, such as snappy or LZ4
    * http://java-performance.info/performance-general-compression/
* Use a diffent big data approach to find a more efficient way to estimating the linear model coefficients:
    * Spark MLLib
    * Dask GLM
    * Turi/Graphlab Create

## Bibliography

* Dask Documentation, http://dask.pydata.org/en/latest/
* Distributed Optimization and Statistical Learning via the Alternating Direction Method of Multipliers, Boyd, et al http://stanford.edu/~boyd/papers/pdf/admm_distr_stats.pdf
* https://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp
* Variable Descriptions: http://stat-computing.org/dataexpo/2009/the-data.html
* Dask example using airline data https://jcrist.github.io/dask-sklearn-part-3.html

## Appendices

### Appendix A - CSV to Parquet Conversion

### Appendix B - Benchmark Tests

### Appendix C - Comparison of Dask Files
* Ryan's Hardware: 
    - CPU: Intel i5-4300M @ 2.60GHz
    - Disk: Samsung SSD 850 Pro
    - RAM: 8 GB
    

* Dask using original csv:
    - no conversion
    - size on disk
        - 11.2 gb
    - benchmark of describing 'Distance':
        - Approx. 4 minutes
* Dask using uncompressed parquet: 
    - conversion to parquet
        - approx 10 minutes
    - size on disk:
        - 13.8 gb
    - benchmark of describing 'Distance':
        - 1 loop, best of 3: 6.2 s per loop
* Dask using gzip compressed parquet:
    - converstion to parquet
        - approx 42 minutes
    - size on disk:
        - 1.36 gb <- big difference
    - benchmark of describing 'Distance':
        - 1 loop, best of 3: 8.83 s per loop

#### Summary
Dask allows for out of core management of data sets. CSV files are universal, but slow to process. Converting to parquet file format, speeds up the process by a factor of 38. Using the gzip compression, reduces size on disk from 13.8gb to 1.36 or about 10% of the uncompressed size. This comes in handy for a distributed processing in a cluster since not as much network bandwidth would be needed. The trade off of compression is a 42.4% increasing in processing time, however, 3 additional seconds is hardly noticable, but might be more of an issue for other tasks. 